In [1]:
# ============================================
# 📘 PHASE 2: PREPROCESS VECTORS → CLUSTERS + FEATURES
# ============================================

# --- Step 1. Imports and Configuration ---
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, LineString, Polygon
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

print("✅ Libraries loaded successfully.")


# --- Step 2. Load Raw Data ---
roads = gpd.read_file("../data_raw/roads.geojson")
power = gpd.read_file("../data_raw/power_lines.geojson")

print(f"Loaded {len(roads)} road features and {len(power)} powerline features.")


# --- Step 3. Project and Clean Geometry ---
roads = roads.to_crs(4326)
power = power.to_crs(4326)

roads = roads.explode(index_parts=False).reset_index(drop=True)
power = power.explode(index_parts=False).reset_index(drop=True)

roads = roads[roads.is_valid]
power = power[power.is_valid]

print("✅ Geometry cleaned and projected to EPSG:4326.")


# --- Step 4. Compute Basic Spatial Attributes ---
roads["length_km"] = roads.geometry.length * 111
power["length_km"] = power.geometry.length * 111

print("✅ Computed lengths (km) for all features.")


# --- Step 5. Extract Centroids for Clustering ---
roads["centroid"] = roads.geometry.centroid
road_points = np.array([[p.x, p.y] for p in roads["centroid"]])


# --- Step 6. KMeans Clustering ---
n_clusters = 20  # adjustable parameter (try 10–50 depending on scale)
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
roads["cluster_id"] = kmeans.fit_predict(road_points)

# Create cluster centroids GeoDataFrame
cluster_df = pd.DataFrame(kmeans.cluster_centers_, columns=["lon", "lat"])
cluster_gdf = gpd.GeoDataFrame(
    cluster_df,
    geometry=gpd.points_from_xy(cluster_df.lon, cluster_df.lat),
    crs="EPSG:4326"
)
cluster_gdf["cluster_id"] = range(n_clusters)  # ✅ fix for merge key

print(f"✅ Generated {n_clusters} spatial clusters across road network.")


# --- Step 7. Derive Cluster-Level Summaries ---
cluster_summary = (
    roads.groupby("cluster_id")
    .agg({
        "length_km": "sum"
    })
    .reset_index()
    .rename(columns={"length_km": "total_road_km"})
)

cluster_gdf = cluster_gdf.merge(cluster_summary, on="cluster_id", how="left")

print("✅ Computed cluster-level summaries.")


# --- Step 8. Fix geometry and Save Outputs ---
# Drop centroid geometry so only one geometry column is written
roads = roads.set_geometry("geometry")
roads = roads.drop(columns=["centroid"])

# Save outputs
roads.to_file("../data_processed/roads_clustered.geojson", driver="GeoJSON")
power.to_file("../data_processed/power_lines_clean.geojson", driver="GeoJSON")
cluster_gdf.to_file("../data_processed/road_clusters.geojson", driver="GeoJSON")

print("💾 Saved:")
print(" - data_processed/roads_clustered.geojson")
print(" - data_processed/power_lines_clean.geojson")
print(" - data_processed/road_clusters.geojson")


# --- Step 9. Validation Summary ---
print("\n📊 Validation Summary")
print(f"Number of clusters: {len(cluster_gdf)}")
print(f"Average road length per cluster: {cluster_gdf['total_road_km'].mean():.2f} km")
print(f"Total road length: {cluster_gdf['total_road_km'].sum():,.2f} km")

print("\n🎯 Phase 2 preprocessing complete.")

✅ Libraries loaded successfully.
Loaded 725553 road features and 1475 powerline features.
✅ Geometry cleaned and projected to EPSG:4326.
✅ Computed lengths (km) for all features.
✅ Generated 20 spatial clusters across road network.
✅ Computed cluster-level summaries.
💾 Saved:
 - data_processed/roads_clustered.geojson
 - data_processed/power_lines_clean.geojson
 - data_processed/road_clusters.geojson

📊 Validation Summary
Number of clusters: 20
Average road length per cluster: 24684.40 km
Total road length: 493,688.00 km

🎯 Phase 2 preprocessing complete.
